In [1]:
import pandas as pd
import json

In [46]:
results = {}
projects = ["JPetStore", "DayTrader", "AcmeAir", "Plants"]
methods = ["Mo2oM", "Mono2Multi", "Mo2oM_HardClustering", "Mo2oM_TFIDF"]
method_names = {"Mo2oM": r"\textbf{UniXcoder + GNN Soft (ours)}",
				"Mono2Multi": "UniXcoder + Distance Soft",
				"Mo2oM_HardClustering": "UniXcoder + GNN Hard",
				"Mo2oM_TFIDF": "TF-IDF + GNN Soft"}
metrics = ["SM", "ICP", "IFN", "NED"]
metrics_sign = {"SM": 1, "ICP": -1, "IFN": -1, "NED": -1}
for project in projects:
	results[project] = {}
	for method in methods:
		results[project][method] = {}
		with open(f"../results/{method}/{method}_{project}.json", "r") as f:
			data = json.load(f)
			df = pd.DataFrame(data)
			for metric in metrics:
					# if metrics_sign[metric] == 1:
					# 	results[project][method][metric] = df[metric].max()
					# else:
					# 	results[project][method][metric] = df[metric].min()
					results[project][method][metric] = df[metric].mean()

In [47]:
bests = []
for project in results:
	for metric in metrics:
		best_method = None
		best_metric = float('-inf') * metrics_sign[metric]
		for method in methods:
			if metrics_sign[metric] == 1:
				if results[project][method][metric] > best_metric:
					best_metric = results[project][method][metric]
					best_method = method
			else:
				if results[project][method][metric] < best_metric:
					best_metric = results[project][method][metric]
					best_method = method
		bests.append((project, metric, best_method))

In [48]:
bests

[('JPetStore', 'SM', 'Mo2oM_TFIDF'),
 ('JPetStore', 'ICP', 'Mo2oM'),
 ('JPetStore', 'IFN', 'Mo2oM'),
 ('JPetStore', 'NED', 'Mo2oM_HardClustering'),
 ('DayTrader', 'SM', 'Mo2oM_TFIDF'),
 ('DayTrader', 'ICP', 'Mo2oM_TFIDF'),
 ('DayTrader', 'IFN', 'Mo2oM'),
 ('DayTrader', 'NED', 'Mono2Multi'),
 ('AcmeAir', 'SM', 'Mono2Multi'),
 ('AcmeAir', 'ICP', 'Mo2oM'),
 ('AcmeAir', 'IFN', 'Mo2oM'),
 ('AcmeAir', 'NED', 'Mo2oM_HardClustering'),
 ('Plants', 'SM', 'Mono2Multi'),
 ('Plants', 'ICP', 'Mo2oM'),
 ('Plants', 'IFN', 'Mo2oM_TFIDF'),
 ('Plants', 'NED', 'Mono2Multi')]

In [49]:
table_head = r'''	\begin{table}[ht]
		\centering
		\begin{tabular}{|l|c|c|c|c|}
			\hline
			\rowcolor{gray!20} % Now light gray (previously our method's color)
			\rule{0pt}{14pt}
			\textbf{Method} & \textbf{SM $\uparrow$} & \textbf{ICP $\downarrow$} & \textbf{IFN $\uparrow$} & \textbf{NED $\downarrow$} \\'''
table_tail = r'''		\end{tabular}
	\end{table}'''

In [50]:
print(table_head)
for project in projects:
	print(r"\hline")
	print(f"\\multicolumn{{5}}{{|c|}}{{\\multirow{{2}}{{*}}{{\\textbf{{{project}}}}}}} \\\\") 
	print(r"\multicolumn{5}{|c|}{} \\")
	print(r"\hline")
	for method in reversed(methods):
		line = method_names[method]
		for metric in metrics:
			if (project, metric, method) in bests:
				line += f" & \\textbf{{{results[project][method][metric]:.3f}}}"
			else:
				line += f" & {results[project][method][metric]:.3f}"
		line += r"\\"
		if method == "Mo2oM":
			print(r"\hline")
			print(r"\rowcolor{yellow!15}")
			print(line)
			print(r"\hline")
		else:
			print(line)
print(table_tail)

	\begin{table}[ht]
		\centering
		\begin{tabular}{|l|c|c|c|c|}
			\hline
			\rowcolor{gray!20} % Now light gray (previously our method's color)
			\rule{0pt}{14pt}
			\textbf{Method} & \textbf{SM $\uparrow$} & \textbf{ICP $\downarrow$} & \textbf{IFN $\uparrow$} & \textbf{NED $\downarrow$} \\
\hline
\multicolumn{5}{|c|}{\multirow{2}{*}{\textbf{JPetStore}}} \\
\multicolumn{5}{|c|}{} \\
\hline
TF-IDF + GNN Soft & \textbf{0.169} & 0.125 & 0.430 & 0.563\\
UniXcoder + GNN Hard & 0.063 & 0.459 & 1.458 & \textbf{0.304}\\
UniXcoder + Distance Soft & 0.129 & 0.373 & 0.768 & 0.320\\
\hline
\rowcolor{yellow!15}
\textbf{UniXcoder + GNN Soft (ours)} & 0.148 & \textbf{0.109} & \textbf{0.312} & 0.541\\
\hline
\hline
\multicolumn{5}{|c|}{\multirow{2}{*}{\textbf{DayTrader}}} \\
\multicolumn{5}{|c|}{} \\
\hline
TF-IDF + GNN Soft & \textbf{0.377} & \textbf{0.107} & 0.435 & 0.638\\
UniXcoder + GNN Hard & 0.087 & 0.458 & 1.544 & 0.643\\
UniXcoder + Distance Soft & 0.170 & 0.441 & 0.855 & \textbf{0.320}\\
\h